In [ ]:
from pycalphad import Database, equilibrium, variables as v, calculate
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)

Компоненты и фазы

In [ ]:
import random

random.seed(10)
all_components = list(db.elements)
all_phases =  list(db.phases)
all_phases.sort()
all_components.sort()
print('Количество фаз =' + str(len(all_phases)))
print('Количество компонент =' + str(len(all_components)))
print(all_components)
# components = random.sample(all_components, 8)
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']

print(components)


Бинарные фазы для компонент

In [ ]:
counter = 0 
components_phases = {}
for comp in components:
    components_phases[comp] = set()
for item in db.phases.items():
    # item[0] - name, item[1] - content
    # item[1] - name, constituents, sublattices, model_hints
    if len(item[1].constituents)>4:
        counter += 1
    
    for species in item[1].constituents[0]:
        # name, constituents, charge
        for phase in components_phases.keys():
            if phase in species.name:
                components_phases[phase].add(item[0])

print(counter)
print(components_phases)

for comp, phases in components_phases.items():
    mask = [1 if phase in phases else 0 for phase in all_phases]
    components_phases[comp] = int(''.join(map(str, mask)), 2)

In [ ]:
for comp, phases in components_phases.items():
    print(f'{comp}: {bin(phases)}')

In [ ]:
def getPhases(intMask):
    return [all_phases[i] for i in range(len(all_phases)) if (intMask & (1 << i)) > 0]
print(getPhases(components_phases['SE']))

In [ ]:
for comp, phases in components_phases.items():
    print(comp)
    print(getPhases(phases))
    for phase in getPhases(phases):
        if str(phase) in db.symbols:
            print(f'{phase} in {db.symbols}')

Функции

In [ ]:
import re

# read the text file
text = ""
function_start = False
with open(db_name, 'r') as f:
    for line in f:
        if "Standard functions for the elements" in line:
            function_start = True
        if "Data for the elements from unary database" in line:
            function_start = False

        if function_start:
            text += line

component_functions = {}
# find all the component names in the text
component_names = re.findall(r'\$ ([A-Z][a-z]*)', text)
for component in component_names:
    # find the text between the component name and the next component name
    pattern = r'\$ {}.*?\$ [A-Z][a-z]*'.format(component)
    component_text = re.search(pattern, text, re.DOTALL).group()
    
    # find all the function names in the component text
    function_names = re.findall(r'FUNCTION ([A-Z]*)', component_text)
    
    component_functions[component] = function_names
component_functions = {k.upper():v for k,v in component_functions.items()}

In [ ]:
# print the functions for each component
for component, functions in component_functions.items():
    if len(functions) > 0:
        print(f"{component} : {functions}")

In [53]:
functions = set()
for component in components:
    if str(component) in component_functions.keys():
        functions.update(component_functions[component])
    else:
        print(f'Check - {component}')
print(functions)

{'GBCCL', 'GLIQAU', 'GHCPAU', 'GLIQGD', 'GHSERVZ', 'GLIQRH', 'GLIQV', 'U', 'GHSERV', 'GB', 'GBCCAU', 'GBCCRH', 'GLIQSE', 'GLIQAL', 'GBCCMZ', 'GFCCGD', 'GFCCZR', 'GFCCHG', 'GHCPHG', 'GLIQZR', 'GHSERAU', 'GBCCAL', 'GHSERSE', 'GHSERGD', 'GRHOMBHG', 'GLAVAL', 'GHSERHG', 'GBCCZR', 'GHSERRH', 'GBCCCZ', 'GHSERAL', 'LB', 'GHCPAL', 'GBCCGD', 'GFE', 'GFENI', 'GHSERZR', 'GHCPRH'}


In [34]:
for func in functions:
    if func in db.symbols:
        print(db.symbols[func])


Piecewise((12552.0 + GHSERAU - 9.385866*T, And(T < 3200.0, 298.15 <= T)), (0, True))
Piecewise((240.75 + GHSERAU + 1.6*T, And(T < 3200.0, 298.15 <= T)), (0, True))
Piecewise((6225.4407 + 88.80921026*T - 24.7214131*T*log(T) - 8665.73348*T**(-1.0) - 0.00285240521*T**2.0 - 3.14674076e-07*T**3.0, And(T < 1000.0, 100.0 <= T)), (146262.037 - 1208.70685*T + 159.352082*T*log(T) - 19678357.0*T**(-1.0) - 0.108247135*T**2.0 + 1.06945505e-05*T**3.0, And(T < 1508.15, 1000.0 <= T)), (-5397.314 + 192.336215*T - 38.5075*T*log(T), And(T < 3600.0, 1508.15 <= T)), (0, True))
Piecewise((-7930.43 + 133.346053*T - 24.134*T*log(T) + 69460.0*T**(-1.0) - 0.003098*T**2.0 + 1.2175e-07*T**3.0, And(T < 790.0, 298.15 <= T)), (-7967.842 + 143.291093*T - 25.9*T*log(T) + 6.25e-05*T**2.0 - 6.8e-07*T**3.0, And(T < 4000.0, 790.0 <= T)), (0, True))
Piecewise((11244.082 + 125.099593*T - 24.0178336*T*log(T) + 55846.0*T**(-1.0) - 0.003424186*T**2.0 - 1.68032e-07*T**3.0, And(T < 700.0, 298.15 <= T)), (35898.508 - 147.926418*T

In [56]:
from symengine.lib.symengine_wrapper import Piecewise, Symbol
import math

T = Symbol('T')
# print(db.symbols['GBCCGD'])
print(db.symbols['GBCCGD'].subs(T, 800))
# print (math.log(14))


Piecewise((70417.066733688 - 19777.1304*log(800), True), (-1024901.6494864 + 144077.6752*log(800), False), (145993.98144 - 31168.34*log(800), False), (161778.079997987 - 33523.4664*log(800), False), (0, True))
